# Input Guardrails in Pydantic AI via Tool Calls

We implemented input guardrails with OpenAI Agents SDK in the previous lesson.

Now we will do the same with PydanticAI.

PydanticAI does not have built-in guardrail functionality like Agents SDK.

We need to implement it ourselves.

There is an open issue that discusses guardrails implementation: https://github.com/pydantic/pydantic-ai/issues/1197

The issue describes two approaches for implementing guardrails:

- Using a tool call to validate input
- Running validation in parallel like Agents SDK does

The tool call approach is simpler to implement.

We will start with that approach.

## Defining the Guardrail Output Model

Create the output model for the guardrail decision. (`guardrails-pydantic-ai/search_agent.py`)


In [ ]:
class EvidentlyDocsGuardrail(BaseModel):
    resoning: str
    fail: bool


## Implementing the Guardrail Tool

Create a tool function that checks for prohibited topics.

The agent will call this tool to validate user input.

In [ ]:
def input_guardrail(message: str) -> EvidentlyDocsGuardrail:
    """
    This function checks if the user message contains prohibited topics.
    Args:
        message: The user input message
    Returns:
        EvidentlyDocsGuardrail indicating if tripwire was triggered           
    """
    prohibited_topics = [
        "sqrt", "math", "history"
    ]

    for topic in prohibited_topics:
        if topic in message.lower():
            return EvidentlyDocsGuardrail(
                reasoning=f'Input contains prohibited topic: {topic}',
                fail=True
            )

    return EvidentlyDocsGuardrail(
        reasoning='Input is clean',
        fail=False
    )


## Adding the Guardrail to the Agent

Add the guardrail function to the agent tools list.

The agent will now have access to the guardrail as a callable tool.

In [ ]:
agent = Agent(
    name="search",
    instructions=search_instructions,
    tools=[
        tools.search,
        tools.read_file,
        input_guardrail
    ],
    model=config.model,
    output_type=SearchResultArticle,
    history_processors=[force_answer_after_6_searches]
)


## Testing the Guardrail

Run the agent with an input that contains a prohibited topic.

The agent should call the guardrail tool and detect the violation.


In [ ]:
result = await run_stream(agent, 'how much is sqrt(pi)', SearchResultHandler())

## Limitations and Next Steps

This approach is simple and works well.

However, it adds to the execution time because the guardrail runs as a tool call. Also, the agent needs to decide to call the guardrail tool first - and this is not guaranteed.

In the next lesson we will implement parallel guardrails.